In [78]:
import numpy as np
from tqdm import tqdm
import re
import pickle
from multiprocessing import Pool
from tqdm import tqdm
from rdkit import Chem
from rdkit import RDLogger
from rdkit.Chem import Draw
from rdkit.Chem.inchi import MolFromInchi
from multiprocessing import Pool
import requests
import pandas as pd

RDLogger.DisableLog('rdApp.*')

In [2]:
with open('./data/CID-InChI-Key', 'r') as f :
    lines = [f.readline() for _ in tqdm(range(2000000))]
    list_InChIs = list(map(lambda line: re.split("\s+", line.strip())[-1], lines))
    
np.random.seed(1)
np.random.shuffle(list_InChIs)

100%|██████████| 2000000/2000000 [00:02<00:00, 843528.46it/s]


In [3]:
list_InChIs[:5]

['UQUUATCGBYRDHH-IBGZPJMESA-N',
 'KHCUXFYAUCRPAZ-UHFFFAOYSA-N',
 'WOVKPADOHXADOO-UHFFFAOYSA-N',
 'KZDKGCIZUZRVHM-CYBMUJFWSA-N',
 'YSQRIFMZXDONSU-UHFFFAOYSA-N']

In [20]:
r = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/UQUUATCGBYRDHH-IBGZPJMESA-N/JSON').json()

In [24]:
r

{'PC_Compounds': [{'id': {'id': {'cid': 2139488}},
   'atoms': {'aid': [1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50,
     51,
     52],
    'element': [35,
     8,
     8,
     8,
     7,
     7,
     7,
     7,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     6,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1,
     1]},
   'bonds': {'aid1': [1,
     2,
     3,
  

In [83]:
def get_key2inchi(key): 
    r = requests.get(f'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/inchikey/{key}/JSON').json()
    for props in r['PC_Compounds'][0]['props'] :
        if props['urn']['label'] == 'InChI' :
            return props['value']['sval']

In [84]:
print(get_key2inchi('UQUUATCGBYRDHH-IBGZPJMESA-N'))

InChI=1S/C21H23BrN4O3/c22-16-10-5-3-8-14(16)20(28)23-13-7-1-2-12-18(27)25-26-19-15-9-4-6-11-17(15)24-21(19)29/h3-6,8-11,19,26H,1-2,7,12-13H2,(H,23,28)(H,24,29)(H,25,27)/t19-/m0/s1


In [85]:
raw_data = {'image_id' : [], 'InChI' : []}
data = pd.DataFrame(raw_data)
data

,image_id,InChI


In [87]:
for i in tqdm(range(10)) :
    key = list_InChIs[i]
    inchi = get_key2inchi(key)
    image_id = str(i).zfill(7)
    data.loc[i] = [image_id, inchi]

100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [88]:
data.head()

,image_id,InChI
0,0000000,InChI=1S/C21H23BrN4O3/c22-16-10-5-3-8-14(16)20...
1,0000001,InChI=1S/C14H8ClF4NO2/c15-8-5-9-11(10(16)6-8)2...
2,0000002,InChI=1S/C16H10BrN3OS2/c17-10-5-9(6-18-7-10)15...
3,0000003,InChI=1S/C15H13N5OS/c1-21-11-6-4-10(5-7-11)13-...
4,0000004,InChI=1S/C16H24N2O2/c1-20-15-8-6-7-14(13-15)17...


In [89]:
for i in tqdm(range(5)) : 
    mol_data = data.iloc[i, :]
    mol = MolFromInchi(mol_data['InChI'])
    img = Draw.MolToImage(mol, size = (224, 224))
    img.save('./data/new/{}.png'.format(mol_data['image_id']))

100%|██████████| 5/5 [00:00<00:00, 106.73it/s]
